In [13]:
import numpy as np
import random
import json
import numpy as np
import random
import json
import time
import json
import base64
import jwt
import pyshark
from jwcrypto import jwk
import jwt
import warnings
warnings.filterwarnings("ignore")
import gym_maze
import requests
import numpy as np
from PIL import Image
from gym_maze.envs.maze_manager import MazeManager
import time
import json
import warnings
warnings.filterwarnings("ignore")
import gym_maze
from gym_maze.envs.maze_manager import MazeManager
rids_solved = []
paths =[]
rid_time = {}
G_moves = {}
prev_acc = []
steps=[]
#import easyocr
#reader = easyocr.Reader(['en'])
    
history = [[[] for j in range(10)] for i in range(10)]

def search_hist(av_actions, block_axis): # if available actions more than one, then pick the oldest action 
    
    
    
    x = block_axis[0]
    y = block_axis[1]
    
    for i in history[x][y]:
        if i not in av_actions:
            history[x][y].remove(i)
    
    oldest = None
    
    # searching if an action not in history
    for ac in av_actions:
        if ac not in history[x][y]:
            oldest = ac
    
    if oldest != None:
        history[x][y].insert(0, oldest)
        return oldest
        
    # bring the oldest to the beginning
    oldest = history[x][y].pop()
    history[x][y].insert(0, oldest)
    
    return oldest


def decode_jwt(token1):
    decoded = jwt.decode(token1, options={"verify_signature": False, "verify_aud": False, "audience": "my_audience"})
    iss1 = decoded.get("iss", "")
    aud1 = decoded.get("aud", "")
    id1 = decoded.get("id", "")
    scope1 = decoded.get("scope", "")
    name1 = decoded.get("name", "")
    email1 = decoded.get("email", "")
    rand1 = decoded.get("rand", "")
    return iss1, aud1, id1, scope1, name1, email1, rand1


def split_binary(binary_num, chunk_size):
    # Make sure the binary number is a string
    binary_str = str(binary_num)
    # Split the binary string into chunks of the given size
    binary_chunks = [binary_str[i:i + chunk_size] for i in range(0, len(binary_str), chunk_size)]

    return binary_chunks


def cipher_solver(encoded_string):
    # Add padding characters to the input string
    padding = len(encoded_string) % 4
    if padding > 0:
        encoded_string += '=' * (4 - padding)

    decoded_bytes = base64.b64decode(encoded_string)
    decoded_string = decoded_bytes.decode('utf-8')

    binary_str = ''
    number_str = ''
    for char in decoded_string:
        if char.isdigit():
            number_str += char
        elif char == ',':
            binary_str += number_str
            number_str = ''

    tuple_str = '(' + binary_str + ',' + number_str + ')'

    split_tuple = tuple_str[1:-1].split(',')

    binary_str = split_tuple[0]
    integer_val = int(split_tuple[1])
    decimal_val = int(str(integer_val), 2)
    length = len(binary_str)
    parts=int(length/decimal_val)

    binary_num = binary_str   # first input after base64 decode
    chunk_size = parts
    binary_chunks = split_binary(binary_num, chunk_size)
    output_string = ' '.join(binary_chunks)

    binary_chunks = [binary_num[i:i+parts] for i in range(0, len(binary_num), parts)]

    decimal_nums = []
    for chunk in binary_chunks:
        decimal_nums.append(int(chunk, 2))


    new_list = []

    for item in decimal_nums:
        if item >= 65 and item <= 90:
            item -= decimal_val
            if item >= 65 and item <= 90:
                new_list.append(item)
            else:
                item += decimal_val+ decimal_val + 8
                new_list.append(item)
        elif item >= 97 and item <= 122:
            item -= decimal_val
            if item >= 97 and item <= 122:
                new_list.append(item)
            else:
                item += decimal_val+ decimal_val + 8
                new_list.append(item)
    ascii_list = new_list
    char_list = []

    for ascii_num in ascii_list:
        char_list.append(chr(ascii_num))
    final=''.join(char_list)
    return final


def captcha_solver(mylist):
    pass
#     t1 = time.time()
#     my_array = np.array(mylist)
#     captcha_image = Image.fromarray(my_array.astype('uint8'))
#     captcha_image = np.array(captcha_image)
#     captcha_image = captcha_image[ :, ::].copy() 
#     # Convert the numpy array to a Pillow image
    
#     # Use pytesseract to convert the image to plain text
#     result = reader.readtext(captcha_image)

#     # Print the plain text
#     t2 = time.time()
#     print(t2-t1)
#     return  result[0][-2]


def pcap_solver(stri):
    
    decoded_data = base64.b64decode(stri)
    
    with open('./output.bin', 'wb') as f:
        f.write(decoded_data)

    capture = pyshark.FileCapture('./output.bin')

    packet_str = ""  # Initialize an empty string to store the packets

    for packet in capture:
        # Append the packet summary to the packet string
        packet_str += str(packet) + "\n"

    capture.close()

    pattern = "google.com:"

    found_lines = set()
    list=[]
    newlist=[None]*4


    for line in packet_str.splitlines():
        if pattern in line and line not in found_lines:
            found_lines.add(line)
            domain = line.split()[0]  # extract the first word (domain name)
            subdomain = domain.split('.')[1]  # extract the second-to-last part of the domain name
            list.append(subdomain)

    newlist[0]=list[3]
    newlist[1]=list[2]
    newlist[2]=list[0]
    newlist[3]=list[1]

    decoded_list = []

    for encoded_str in newlist:
        decoded_bytes = base64.b64decode(encoded_str)
        decoded_str = decoded_bytes.decode('utf-8')
        decoded_list.append(decoded_str)



    final_string = ''.join(decoded_list)
    return final_string

def server_solver(token1):
    
    with open("keypair.pem", "rb") as pemfile:
        key = jwk.JWK.from_pem(pemfile.read())
        public_key = key.export(private_key=False)

    jwks = {
        "e": key['e'],
        "kid": key.key_id,
        "kty": "RSA",
        "n": key['n']
    }

    header = {
        "kid": key.key_id,
        "typ": "JWT",
        "alg": "RS256",
        "jwk": jwks
    }

    fields = decode_jwt(token1)

    iss1, aud1, id1, scope1, name1, email1, rand1 = fields

    claims = {
        "iss": iss1,
        "aud": aud1,
        "id": id1,
        "scope": scope1,
        "name": name1,
        "email": email1,
        "admin": "true",
        "rand": rand1
    }
    key_pem = key.export_to_pem(private_key=True, password=None)
    signed_token = jwt.encode(payload=claims, key=key_pem, algorithm='RS256', headers=header)
    
    return signed_token.decode('utf-8')

rev_actions = {"N":"S","S":"N","E":"W","W":"E"}
# Loop through all possible points
for i in range(10):
    for j in range(10):
        # Create a tuple of the current point
        point = f"[{str(i)} {str(j)}]"
        # Set the value of the dictionary to ["N","S","E","W"]
        G_moves[str(point)] = ["N","S","E","W"]
        if j ==9:
            G_moves[str(point)].remove("S")
        if i ==9:
            G_moves[str(point)].remove("E")
        if j ==0:
            G_moves[str(point)].remove("N")
        if i ==0:
            G_moves[str(point)].remove("W")



# elemnate dead ends 
def elemenate_deadEnd(action):
    c_point = paths[-1]
    
    if(action =='N'):
        rev = 'S'
        next_point =  c_point[:3]+ str(int(c_point[3])-1)+c_point[4:]
    elif(action =='S'):
        rev = 'N'
        next_point =  c_point[:3]+ str(int(c_point[3])+1)+c_point[4:]
    elif(action =='W'):
        rev = 'E'
        next_point =  c_point[:1]+ str(int(c_point[1])-1)+c_point[2:]
    elif(action =='E'):
        rev = 'W'
        next_point =  c_point[:1]+ str(int(c_point[1])+1)+c_point[2:]
    G_moves[next_point].remove(rev)

#save last action
def save_action(action):
    prev_acc.append(action)

# check if we hit a wall 
def check_point():
    if paths[-1] == paths[-2]:
        
        G_moves[paths[-1]].remove(prev_acc[-1])
        
        i = int(paths[-1][1])
        j = int(paths[-1][3])
        if(prev_acc[-1] =='N'):
            j-=1
            G_moves[f"[{i} {j}]"].remove("S")

        elif(prev_acc[-1] =='S'):
            j+=1
            G_moves[f"[{i} {j}]"].remove("N")
           
        elif(prev_acc[-1] =='W'):
            i-=1
            G_moves[f"[{i} {j}]"].remove("E")
            
        elif(prev_acc[-1] =='E'):
            i+=1
            G_moves[f"[{i} {j}]"].remove("W")
        


def select_action(state):
    actions = ['N', 'S', 'E', 'W']
    paths.append(str(state[0])) 
    if len(paths) >=2:
        check_point()
        available_values = G_moves[str(state[0])]
        random_action =random.choice(available_values)
        #elemnate dead ends
        if len(available_values) == 1:
            elemenate_deadEnd(random_action)
            
        else:
            random_action = search_hist(available_values,state[0])
       
        save_action(random_action)
        action_index = actions.index(random_action)
        return random_action, action_index

    else:
        #first two actions random
        available_values = G_moves[str(state[0])]
        random_action = search_hist(available_values,state[0])
        save_action(random_action)
        action_index = actions.index(random_action)
        return random_action, action_index


def local_inference(riddle_solvers):

    obv = manager.reset(agent_id)
    for t in range(MAX_T):
        # Select an action
        time.sleep(.3)
        state_0 = obv
        action, action_index = select_action(state_0) # Random action
        obv, reward, terminated, truncated, info = manager.step(agent_id, action)
        if not info['riddle_type'] == None:
            if info['riddle_type'] not in rids_solved:
                steps.append(t)
                rids_solved.append(info['riddle_type'])
                solution = riddle_solvers[info['riddle_type']](info['riddle_question'])
                obv, reward, terminated, truncated, info = manager.solve_riddle(info['riddle_type'], agent_id, solution)
        
       
        # THIS IS A SAMPLE TERMINATING CONDITION WHEN THE AGENT REACHES THE EXIT
        # IMPLEMENT YOUR OWN TERMINATING CONDITION
        if (len(rids_solved) ==4):
            print(steps)
            manager.set_done(agent_id)
            break # Stop Agent

        if RENDER_MAZE:
            manager.render(agent_id)
       
        states[t] = [obv[0].tolist(), action_index, str(manager.get_rescue_items_status(agent_id))]       
        


if __name__ == "__main__":

    sample_maze = np.load("hackathon_sample.npy")
    agent_id = "9" # add your agent id here
    manager = MazeManager()
    manager.init_maze(agent_id, maze_cells=sample_maze)
    env = manager.maze_map[agent_id]

    riddle_solvers = {'cipher': cipher_solver, 'captcha': captcha_solver, 'pcap': pcap_solver, 'server': server_solver}
    maze = {}
    states = {}

    
    maze['maze'] = env.maze_view.maze.maze_cells.tolist()
    maze['rescue_items'] = list(manager.rescue_items_dict.keys())
    MAX_T = 5000
    RENDER_MAZE = True
    
    t1 =time.time()
    local_inference(riddle_solvers)
    t2 =time.time()
    
    with open("./states.json", "w") as file:
        json.dump(states, file)

    
    with open("./maze.json", "w") as file:
        json.dump(maze, file)



riddle failed
server
rescue item nulled

riddle failed
captcha
rescue item nulled



AttributeError: type object '_asyncio.Task' has no attribute 'all_tasks'

In [7]:

sample_maze = np.load("hackathon_sample.npy")
agent_id = "9" # add your agent id here
manager = MazeManager()
manager.init_maze(agent_id, maze_cells=sample_maze)
env = manager.maze_map[agent_id]

riddle_solvers = {'cipher': cipher_solver, 'captcha': captcha_solver, 'pcap': pcap_solver, 'server': server_solver}
maze = {}
states = {}

    
maze['maze'] = env.maze_view.maze.maze_cells.tolist()
maze['rescue_items'] = list(manager.rescue_items_dict.keys())
MAX_T = 5000
RENDER_MAZE = True
    
local_inference(riddle_solvers)
    
with open("./states.json", "w") as file:
    json.dump(states, file)

    
with open("./maze.json", "w") as file:
    json.dump(maze, file)


riddle failed
server
rescue item nulled



ValueError: list.remove(x): x not in list

In [151]:
env.cls

In [113]:
history = [[[] for j in range(10)] for i in range(10)]
history

def search_hist(av_actions, block_axis): # if available actions more than one, then pick the oldest action 
    x = block_axis[0]
    y = block_axis[1]
    
    # searching if an action not in history
    for ac in av_actions:
        if ac not in history[x][y]:
            history[x][y].insert(0,ac)
            return ac
        
    # bring the oldest to the beginning
    oldest = history[x][y].pop()
    history[x][y].insert(0, oldest)
    
    return oldest

In [124]:
search_hist(['W', 'E','N'], [0,0])

'W'

In [125]:
history[0][0]

['W', 'N', 'E']